In [1]:
import numpy as np
import yaml
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Step start

$$ v \to v + a \Delta t $$

$$ q = p $$

$$ p \to p + v \Delta t $$

In [2]:
def generate_edge_constraints(p, e):
    return np.sqrt(((p[e[:, 0]] - p[e[:, 1]])**2).sum(axis=1))

def generate_triangle_constraints(p, t):
    a, b, c = p[t[:, 0]], p[t[:, 1]], p[t[:, 2]]
    return np.cross(b - a, c - a)

def step_start(p, q, v, a, dt):
    v += a * dt
    q[:] = p[:]
    p += v * dt

def collide_with_floor(p):
    p[:, 1][p[:, 1] < 0] = 0

def edge_constraint(p, w, e, l0, k=1.):
    """
    Parameters
    ----------
    p: Positions
    w: Weights
    e: Edges
    l0: Rest lengths
    k: Stiffness
    """
    for i in range(len(e)):
        e0, e1 = e[i, 0], e[i, 1]
        p0, p1, w0, w1 = p[e0], p[e1], w[e0], w[e1]
        n = p1 - p0
        l = np.linalg.norm(n)
        if l <= 1e-9:
            continue
        n = -n / l
        lam = -(l - l0[i]) / (w0 + w1)
        dx = k * lam * n
        p[e0] += w0 * dx
        p[e1] -= w1 * dx

def triangle_constraint(p, w, t, a0, k=1.):
    for i in range(len(t)):
        t0, t1, t2 = t[i, 0], t[i, 1], t[i, 2]
        p0, p1, p2, w0, w1, w2 = p[t0], p[t1], p[t2], w[t0], w[t1], w[t2]
        
        p01 = p1 - p0
        p02 = p2 - p0
        p12 = p2 - p1
        lam = w0 * np.dot(p12, p12) + w1 * np.dot(p02, p02) + w2 * np.dot(p01, p01)
        if lam <= 1e-9:
            continue

        lam = k / lam
        a = np.cross(p01, p02)
        if a <= 1e-9:
            continue

        dx = lam * (a - a0[i])
        ccw = np.array([[0, -1], [1, 0]])
        p[t0] += dx * w0 * ccw @ p12
        p[t1] += dx * w1 * ccw @ -p02
        p[t2] += dx * w2 * ccw @ p01

def step_end(p, q, v, dt):
    v[:] = (p[:] - q[:]) / dt

In [3]:
np.random.seed(42)

dt = 1 / 60. / 8
g = np.array([0, -20])

with open('data.yaml') as f:
    data = yaml.safe_load(f)

positions = np.array(data['positions']) + np.array([0, 2])
edges = np.array(data['edges'])
triangles = np.array(data['triangles'])
weights = np.array(data['weights'])

previous_positions = np.zeros(positions.shape)
velocities = np.zeros(positions.shape)
constraint_e = generate_edge_constraints(positions, edges)
constraint_t = generate_triangle_constraints(positions, triangles)

oscilators = np.zeros((len(positions), 3))
oscilators[0] = np.array([0.2, 25, 0])

In [4]:
x = [ positions.copy() ]
t = 0.

gensin = lambda o, e, t: o[e, 0] * np.sin(o[e, 1] * t + o[e, 2])
factor = lambda o, e, t: 1 + gensin(o, e[:, 0], t) + gensin(o, e[:, 1], t)
for i in range(8 * 150):
    step_start(positions, previous_positions, velocities, g, dt)
    f = factor(oscilators, edges, t)
    edge_constraint(positions, weights, edges, f * constraint_e, k=1.)
    triangle_constraint(positions, weights, triangles, constraint_t, k=1.)
    collide_with_floor(positions)
    step_end(positions, previous_positions, velocities, dt)  
    x.append(positions.copy())
    t += dt

In [5]:
def update(i, x, e):
    ax.clear()
    ax.set_aspect('equal')
    ax.set_xlim((-2, +10))
    ax.set_ylim((-0.1, 4))
    ax.scatter(x[i][:, 0], x[i][:, 1], color='blue')
    for ei in e:
        p = x[i][ei]
        ax.plot(p[:, 0], p[:, 1], color='blue')

fig, ax = plt.subplots()

ani = FuncAnimation(fig, update, frames=len(x[::8]), interval=100, fargs=(x[::8],edges,))
ani.save('ani.gif')
plt.close()

# Bibliography

[1] Bringing drawings to life: evolving distributed controllers for hand-drawn
soft-bodied robots
Michał Joachimczak1y, Rishemjit Kaur1;2, Reiji Suzuki1, Takaya Arita1